In [1]:
from test_savant.guard import InputGuard, OutputGuard
from test_savant.guard.output_scanners import BanCode
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def dummy_llm(input):
    return input
long_article = open("../docs/article.txt", "r").read()
print(len(long_article))

ts_api = InputGuard(
    API_KEY=os.environ.get("TEST_SAVANT_API_KEY"),
    PROJECT_ID=os.environ.get("TEST_SAVANT_PROJECT_ID"),
    remote_addr=os.environ.get("TEST_SAVANT_REMOTE_ADDR")
)
scanner = BanCode(tag="base", threshold=0.5)
ts_api.add_scanner(scanner)

7810


In [3]:
prompt = """
import time, os.path
photofiles = ['img_1074.jpg', 'img_1076.jpg', 'img_1077.jpg']
class BatchRename(Template):
    delimiter = '%'

fmt = input('Enter rename style (%d-date %n-seqnum %f-format):  ')


t = BatchRename(fmt)
date = time.strftime('%d%b%y')
for i, filename in enumerate(photofiles):
    base, ext = os.path.splitext(filename)
    newname = t.substitute(d=date, n=i, f=ext)
    print('{0} --> {1}'.format(filename, newname))
"""
prompts = [prompt, prompt + " " + long_article, long_article + " " + prompt]

for prompt in prompts:
    result = ts_api.scan(prompt)
    if result.is_valid:
        print("Prompt is valid")
    else:
        print("Prompt is invalid")

{"prompt": "\nimport time, os.path\nphotofiles = ['img_1074.jpg', 'img_1076.jpg', 'img_1077.jpg']\nclass BatchRename(Template):\n    delimiter = '%'\n\nfmt = input('Enter rename style (%d-date %n-seqnum %f-format):  ')\n\n\nt = BatchRename(fmt)\ndate = time.strftime('%d%b%y')\nfor i, filename in enumerate(photofiles):\n    base, ext = os.path.splitext(filename)\n    newname = t.substitute(d=date, n=i, f=ext)\n    print('{0} --> {1}'.format(filename, newname))\n", "config": {"project_id": "db8374b7-72f4-415d-9e70-2e2ee2afc047", "fail_fast": true, "cache": {"enabled": true, "ttl": 3600}}, "use": [{"name": "BanCode:base", "params": {"threshold": 0.5}, "type": "BanCode"}]}


Exception: Request failed with status code 500